<img src="./lecture_image/00_title.png">

<table border=1 width=100%>
    <tr><td style="border: 1px solid black; width:600px; height:30px; text-align: center;"><font size=4 color=blue><b>[21차시] 학습목표</b></font></td></tr>       
    <tr><td style="border: 1px solid black; text-align: left;"><font size=3>
        
○ Mediapipe 라이브러리를 활용할 수 있다 <br><br>
○ Mediapipe 라이브러리를 이용하여 얼굴 특성을 추출할 수 있다 <br><br>
○ Mediapipe 라이브러리를 이용하여 동작 특성을 추출할 수 있다 </font></td></tr>   
</table>

# Mediapipe 라이브러리를 이용한 얼굴 3D 데이터 추출

- 라이브 및 스트리밍 미디어를 위한 플랫폼 간 사용자 지정 가능한 ML 솔루션을 제공
- mediapipe 라이브러리 : https://google.github.io/mediapipe/getting_started/python
- 기능
<table width=800>
    <tr>
        <td><center><b>얼굴 검출</b></center></td>
        <td><center><b>얼굴 Mesh</b></center></td>
        <td><center><b>홍채 검출</b></center></td>
        <td><center><b>손동작 검출</b></center></td>
        <td><center><b>동작 검출</b></center></td>
        <td><center><b>전체동작</b></center></td>
    </tr>     
    <tr>
        <td><img src="./lecture_image/21_mediapipe_face_detection.gif"></td>
        <td><img src="./lecture_image/21_mediapipe_face_mesh.gif"></td>
        <td><img src="./lecture_image/21_mediapipe_iris_tracking.gif "></td>
        <td><img src="./lecture_image/21_mediapipe_hand_tracking.gif"></td>
        <td><img src="./lecture_image/21_mediapipe_pose_tracking.gif"></td>
        <td><img src="./lecture_image/21_mediapipe_holistic_tracking.gif"></td>
    </tr>    
    <tr>
        <td><center><b>헤어 분리</b></center></td>
        <td><center><b>객체 검출</b></center></td>
        <td><center><b>박스 추적</b></center></td>
        <td><center><b>모션 추적</b></center></td>
        <td><center><b>3D객체 검출</b></center></td>
        <td><center><b>템플릿 매칭</b></center></td>
    </tr> 
    <tr>
        <td><img src="./lecture_image/21_mediapipe_hair_segmentation.gif"></td>        
        <td><img src="./lecture_image/21_mediapipe_object_detection.gif"></td>
        <td><img src="./lecture_image/21_mediapipe_box_tracking.gif"></td>
        <td><img src="./lecture_image/21_mediapipe_instant_motion_tracking.gif"></td>
        <td><img src="./lecture_image/21_mediapipe_objectron_chair.gif"></td>
        <td><img src="./lecture_image/21_mediapipe_template_matching.gif"></td>
    </tr> 
</table>    

In [1]:
# 프롬프트 창에서 설치
!pip install mediapipe

     ---------------------------------------- 49.8/49.8 MB 9.6 MB/s eta 0:00:00


In [3]:
import cv2
import mediapipe as mp

# No module named 'mediapipe'가 뜨는 경우
#   - 설치 폴더 (아래 설치내용 확인) 확인해서 .\anaconda3\envs\gpu\lib\site-packages로 이동

# No module named 'mediapipe.python._framework_bindings가 뜨는 경우
#   - 

## 얼굴 메시 추출

In [4]:
#실습
import cv2
import mediapipe as mp

# 시각화 라이브러리 가져오기
mp_drawing = mp.solutions.drawing_utils
# 얼굴 메쉬를 추출하는 라이브러리
mp_face_mesh = mp.solutions.face_mesh

# 선과 점 그리기 설정
drawing_spec = mp_drawing.DrawingSpec(thickness=1,circle_radius=1)

cap = cv2.VideoCapture('images/face3.mp4')
# 얼굴 메시 추출
# min_detection_confidence = 최소 얼굴 검출 정확도
# min_tracking_confidence = 최소 얼굴 추적 정확도
with mp_face_mesh.FaceMesh(min_detection_confidence=0.5,
                          min_tracking_confidence=0.5) as face_mesh:
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            break
            
        # 메시 추출을 실행하는 코드
        # 처리전에 쓰기기능을 준단하고 처리후에 쓰기기능을 사용하게하면 속도가 향샹
        frame.flags.writeable = False
        results = face_mesh.process(frame)
        frame.flags.writeable = True
        
        # 추출된 랜드마크 점과 점을 연결하는 메쉬를 그려준다
        # 얼굴이 검출되었다면
        if results.multi_face_landmarks :
            # 랜드마크점을 하나씩 가져온다
            for face_landmark in results.multi_face_landmarks:
                # 랜드마크 점을 그리는 함수
                mp_drawing.draw_landmarks(
                    image = frame,
                    landmark_list = face_landmark,
                    # 사용한 알고리즘
                    connections = mp_face_mesh.FACEMESH_TESSELATION,
                    # 랜드마크 점 그리기 설정
                    landmark_drawing_spec = drawing_spec,
                    # 메쉬 그리기 설정
                    connection_drawing_spec = drawing_spec
                )
                
        cv2.imshow('face mesh',frame)
        key = cv2.waitKey(33)
        
        if key == 49 :
            break
cap.release()
cv2.destroyAllWindows()

## 얼굴, 왼손, 오른손, 동작 검출 (스켈레톤)

In [5]:
#실습
import cv2
import mediapipe as mp

# 시각화 라이브러리 가져오기
mp_drawing = mp.solutions.drawing_utils
# 스켈레톤 추출 라이브러리
mp_holistic = mp.solutions.holistic

cap = cv2.VideoCapture('images/face4.mp4')

# 선과 점 그리기 설정
drawing_spec1 = mp_drawing.DrawingSpec(thickness=1,color=(0,0,255))
drawing_spec2 = mp_drawing.DrawingSpec(thickness=3,color=(255,0,0))

with mp_holistic.Holistic(min_detection_confidence=0.5,
                         min_tracking_confidence=0.5) as holistic :
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            break
            
        frame.flags.writeable = False
        results = holistic.process(frame)
        frame.flags.writeable = True
        
        mp_drawing.draw_landmarks(
            frame,
            landmark_list=results.pose_landmarks,
            connections=mp_holistic.POSE_CONNECTIONS,
            landmark_drawing_spec=drawing_spec1,
            connection_drawing_spec=drawing_spec2
        )
        
        cv2.imshow('skeleton',frame)
        key = cv2.waitKey(33)
        if key == 49 :
            break
cap.release()
cv2.destroyAllWindows()

<table border=1 width=100%>
    <tr><td style="border: 1px solid black; width:600px; height:40px; text-align: center;"><font size=4 color=blue><b>[21차시] 정리하기</b></font></td></tr>       
    <tr><td style="border: 1px solid black; text-align: left;"><font size=3>○            
        </font></td></tr>   
</table>